In [1]:
from fastai.collab import *
from fastai.tabular.all import *
path = untar_data(URLs.ML_100k)

In [2]:
path.ls()

(#23) [Path('/root/.fastai/data/ml-100k/u4.base'),Path('/root/.fastai/data/ml-100k/u4.test'),Path('/root/.fastai/data/ml-100k/u.data'),Path('/root/.fastai/data/ml-100k/allbut.pl'),Path('/root/.fastai/data/ml-100k/mku.sh'),Path('/root/.fastai/data/ml-100k/ua.test'),Path('/root/.fastai/data/ml-100k/ua.base'),Path('/root/.fastai/data/ml-100k/u.item'),Path('/root/.fastai/data/ml-100k/u2.test'),Path('/root/.fastai/data/ml-100k/u3.test')...]

In [3]:
ratings = pd.read_csv(path/'u.data', delimiter='\t',header=None,
                      names=['user', 'movie', 'rating', 'timestamp'])
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
movies= pd.read_csv(path/'u.item',  delimiter='|',encoding='latin-1',
                    usecols=(0,1),names=('movie', 'title'), header=None)
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [6]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,186,302,3,891717742,L.A. Confidential (1997)
2,22,377,1,878887116,Heavyweights (1994)
3,244,51,2,880606923,Legends of the Fall (1994)
4,166,346,1,886397596,Jackie Brown (1997)


In [50]:
ratings.rating.unique()

array([3, 1, 2, 4, 5])

In [65]:
ratings.rating = ratings.rating -1
ratings.rating.unique()

array([2, 0, 1, 3, 4])

In [66]:
dls = CollabDataLoaders.from_df(ratings, user_name='user', item_name='title', rating_name='rating', bs=64)
dls.show_batch()

,user,title,rating
0,425,Cliffhanger (1993),2
1,717,"Grass Harp, The (1995)",3
2,267,Home Alone (1990),2
3,846,Unforgiven (1992),4
4,943,Under Siege 2: Dark Territory (1995),2
5,844,Lawrence of Arabia (1962),2
6,125,Sgt. Bilko (1996),2
7,943,With Honors (1994),0
8,96,"Body Snatcher, The (1945)",3
9,301,Dances with Wolves (1990),3


In [12]:
dls.classes['title'][:5], dls.classes['user'][:5]

((#5) ['#na#',"'Til There Was You (1997)",'1-900 (1994)','101 Dalmatians (1996)','12 Angry Men (1957)'],
 (#5) ['#na#',1,2,3,4])

In [76]:
X, Y = dls.one_batch()
X[:5], Y[:5]

(tensor([[ 719, 1330],
         [ 207, 1205],
         [ 145,  450],
         [ 295,  128],
         [ 299,  777]]),
 tensor([[4],
         [2],
         [1],
         [3],
         [3]], dtype=torch.int8))

In [30]:
my_str = '#na#'
first_occurrence_index = ratings[ratings['user'] == my_str].first_valid_index()
print(first_occurrence_index)

None


In [32]:
class CollabNNCEL(Module):
    def __init__(self, user_sz, movie_sz, n_act=100):
        self.user_factors = Embedding(*user_sz)
        self.movie_factors = Embedding(*movie_sz)

        self.layers = nn.Sequential(
            nn.Linear(user_sz[1] + movie_sz[1], n_act),
            nn.ReLU(),
            nn.Linear(n_act, 5),
            nn.Softmax(dim=1),
        )

    
    def forward(self, x):
        embds = self.user_factors(x[:,0]), self.movie_factors(x[:,1])

        x = self.layers(torch.cat(embds, dim=1))
        return x

In [23]:
embs = get_emb_sz(dls)
embs

[(944, 74), (1665, 102)]

In [91]:
model = CollabNNCEL(*embs)

In [77]:
y_pred = model.forward(X)
y_pred.shape, y_pred[0].sum()

(torch.Size([64, 5]), tensor(1., grad_fn=<SumBackward0>))

In [87]:
Y.shape, Y.squeeze().shape, Y.shape[0]

(torch.Size([64, 1]), torch.Size([64]), 64)

In [81]:
arr = [i for i in range(64)]
y_pred[arr,Y.squeeze().long()].shape, y_pred[arr,Y.squeeze().long()].mean()

(torch.Size([64]), tensor(0.1955, grad_fn=<MeanBackward0>))

In [92]:
def loss_func(pred, target):
    arr = [i for i in range(target.shape[0])]
    return pred[arr,target.squeeze().long()].mean()

In [89]:
loss_func(y_pred, Y)

tensor(0.1955, grad_fn=<MeanBackward0>)

In [93]:
learn_CE = Learner(dls, model, loss_func=loss_func)
learn_CE.fit_one_cycle(5, 5e-3, wd=0.01)

epoch,train_loss,valid_loss,time
0,0.039395,0.040351,00:18
1,0.036246,0.039638,00:14
2,0.030472,0.036778,00:15
3,0.030077,0.035654,00:14
4,0.028199,0.035397,00:15


In [94]:
learn_CE.predict??

Signature: learn_CE.predict(item, rm_type_tfms=None, with_input=False)
Docstring: Prediction on `item`, fully decoded, loss function decoded and probabilities
Source:   
    def predict(self, item, rm_type_tfms=None, with_input=False):
        dl = self.dls.test_dl([item], rm_type_tfms=rm_type_tfms, num_workers=0)
        inp,preds,_,dec_preds = self.get_preds(dl=dl, with_input=True, with_decoded=True)
        i = getattr(self.dls, 'n_inp', -1)
        inp = (inp,) if i==1 else tuplify(inp)
        dec = self.dls.decode_batch(inp + tuplify(dec_preds))[0]
        dec_inp,dec_targ = map(detuplify, [dec[:i],dec[i:]])
        res = dec_targ,dec_preds[0],preds[0]
        if with_input: res = (dec_inp,) + res
        return res
File:      /home/sagar/git/fastai/fastai/learner.py
Type:      method

In [99]:
X

tensor([[ 719, 1330],
        [ 207, 1205],
        [ 145,  450],
        [ 295,  128],
        [ 299,  777],
        [ 540,  988],
        [ 788, 1392],
        [ 334,  401],
        [ 429,  923],
        [ 901,  383],
        [ 409,  340],
        [ 125,  946],
        [ 392,  628],
        [ 825, 1385],
        [ 798,  695],
        [ 620, 1407],
        [ 407, 1249],
        [ 774,  134],
        [ 527,  238],
        [ 871, 1206],
        [ 435,  624],
        [ 128,  274],
        [ 479,  748],
        [ 142,  884],
        [ 342,  391],
        [ 121,  777],
        [ 480, 1558],
        [ 360,  973],
        [ 535,  456],
        [ 222, 1202],
        [ 102, 1044],
        [ 481, 1021],
        [ 276,  260],
        [ 213,    8],
        [ 666,   89],
        [ 868, 1156],
        [ 880, 1141],
        [   3,  983],
        [ 911,  748],
        [ 533, 1396],
        [ 508,  115],
        [ 489,  855],
        [ 883,   45],
        [ 542,  447],
        [  16, 1452],
        [ 

In [105]:
data = {
    'user': [425, 717],
    'title': ['Cliffhanger (1993)', 'Grass Harp, The (1995)'],
    'rating': [2, 3]
}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

   user                   title  rating
0   425      Cliffhanger (1993)       2
1   717  Grass Harp, The (1995)       3


In [106]:
dl = learn_CE.dls.test_dl(df)

In [109]:
learn_CE.validate( dl)

TypeError: list indices must be integers or slices, not TabWeightedDL